In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
raw_2019 = pd.read_csv("./survey_results_public_2019.csv")

In [3]:
raw_2019.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [107]:
raw_2019['DevType'].head(20)

0                                                   NaN
1     Developer, desktop or enterprise applications;...
2     Designer;Developer, back-end;Developer, front-...
3                                 Developer, full-stack
4     Academic researcher;Developer, desktop or ente...
5     Data or business analyst;Data scientist or mac...
6                         Designer;Developer, front-end
7        Developer, back-end;Engineer, site reliability
8     Database administrator;Developer, back-end;Dev...
9     Data or business analyst;Data scientist or mac...
10                                                  NaN
11    Data or business analyst;Data scientist or mac...
12    Data or business analyst;Database administrato...
13        Developer, desktop or enterprise applications
14                                              Student
15                                Developer, full-stack
16    Developer, back-end;Developer, front-end;Devel...
17                             Data or business 

In [15]:
languages = raw_2019[['Respondent', 'LanguageWorkedWith']]

In [16]:
b = languages['LanguageWorkedWith'].str.split(';', expand=True)

In [29]:
l = []
for i in range(28):
    for j in b[i].unique():
        l.append(j)

In [32]:
all_lang = list(dict.fromkeys(l))

In [4]:
df_2019 = raw_2019[['Respondent', 'Employment', 'EdLevel', 'UndergradMajor', 'OrgSize','DevType', 'YearsCode', 'YearsCodePro', 'ConvertedComp', 'Age', 'Gender', 'Country']]

In [5]:
df_2019.head()

,Respondent,Employment,EdLevel,UndergradMajor,OrgSize,DevType,YearsCode,YearsCodePro,ConvertedComp,Age,Gender,Country
0,1,"Not employed, and not looking for work",Primary/elementary school,NaN,NaN,NaN,4,NaN,NaN,14.0,Man,United Kingdom
1,2,"Not employed, but looking for work","Secondary school (e.g. American high school, G...",NaN,NaN,"Developer, desktop or enterprise applications;...",NaN,NaN,NaN,19.0,Man,Bosnia and Herzegovina
2,3,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,1,8820.0,28.0,Man,Thailand
3,4,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,"Developer, full-stack",3,Less than 1 year,61000.0,22.0,Man,United States
4,5,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...","10,000 or more employees","Academic researcher;Developer, desktop or ente...",16,9,NaN,30.0,Man,Ukraine


In [6]:
df_2019.shape

(88883, 12)

In [7]:
df_2019 = df_2019[~df_2019['ConvertedComp'].isna()]

In [8]:
avg_by_country = df_2019.groupby(by=['Country'])['ConvertedComp'].mean()

In [9]:
df_2019 = df_2019.merge(avg_by_country, left_on='Country', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'CountryAvgComp'})

In [10]:
avg_by_ed_lvl = df_2019.groupby(by=['EdLevel'])['ConvertedComp'].mean()

In [11]:
df_2019 = df_2019.merge(avg_by_ed_lvl, left_on='EdLevel', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'EducationAvgComp'})

In [12]:
avg_by_major = df_2019.groupby(by=['UndergradMajor'])['ConvertedComp'].mean()

In [13]:
df_2019 = df_2019.merge(avg_by_major, left_on='UndergradMajor', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'MajorAvgComp'})

In [14]:
df_2019.groupby(by=['Gender']).count()

,Respondent,Employment,EdLevel,UndergradMajor,OrgSize,DevType,YearsCode,YearsCodePro,ConvertedComp,Age,Country,CountryAvgComp,EducationAvgComp,MajorAvgComp
Gender,,,,,,,,,,,,,,
Man,50503,50387,49787,46367,49671,49872,50441,50377,50503,48735,50503,50503,49787,46367
"Man;Non-binary, genderqueer, or gender non-conforming",115,115,111,112,113,114,115,115,115,113,115,115,111,112
"Non-binary, genderqueer, or gender non-conforming",342,341,334,300,333,335,339,338,342,318,342,342,334,300
Woman,3762,3754,3701,3500,3660,3703,3759,3747,3762,3642,3762,3762,3701,3500
Woman;Man,40,40,39,35,40,39,40,40,40,35,40,40,39,35
"Woman;Man;Non-binary, genderqueer, or gender non-conforming",25,24,22,16,24,24,22,22,25,22,25,25,22,16
"Woman;Non-binary, genderqueer, or gender non-conforming",106,106,106,88,103,105,104,106,106,105,106,106,106,88


In [15]:
df_2019['Gender'] = df_2019['Gender'].fillna('Not Specified')

In [16]:
le = preprocessing.LabelEncoder()
le.fit(list(df_2019['Gender'].unique()))

LabelEncoder()

In [17]:
df_2019['GenderCode'] = le.transform(df_2019['Gender'])

In [18]:
df_2019.loc[df_2019['OrgSize']=='1,000 to 4,999 employees', 'NewOrgSize'] = 3000
df_2019.loc[df_2019['OrgSize']=='10 to 19 employees', 'NewOrgSize'] = 15
df_2019.loc[df_2019['OrgSize']=='10,000 or more employees', 'NewOrgSize'] = 10000
df_2019.loc[df_2019['OrgSize']=='100 to 499 employees', 'NewOrgSize'] = 300
df_2019.loc[df_2019['OrgSize']=='2-9 employees', 'NewOrgSize'] = 5
df_2019.loc[df_2019['OrgSize']=='20 to 99 employees', 'NewOrgSize'] = 60
df_2019.loc[df_2019['OrgSize']=='5,000 to 9,999 employees', 'NewOrgSize'] = 7500
df_2019.loc[df_2019['OrgSize']=='500 to 999 employees', 'NewOrgSize'] = 750
df_2019.loc[df_2019['OrgSize']=='Just me - I am a freelancer, sole proprietor, etc.', 'NewOrgSize'] = 1

In [19]:
orgSizeMean = df_2019['NewOrgSize'].mean()

In [20]:
df_2019['NewOrgSize'] = df_2019['NewOrgSize'].fillna(value=orgSizeMean)

In [21]:
avgComp = df_2019['ConvertedComp'].mean()

In [22]:
df_2019['EducationAvgComp'] = df_2019['EducationAvgComp'].fillna(value=avgComp)

In [23]:
df_2019['MajorAvgComp'] = df_2019['MajorAvgComp'].fillna(value=avgComp)

In [24]:
df_2019 = df_2019.reset_index().drop('index', axis=1)

In [26]:
temp1 = df_2019['DevType'].str.split(';', expand=True)

In [28]:
temp1.shape

(55823, 24)

In [30]:
all_positions = []
for row in range(temp1.shape[0]):
    j = temp1.iloc[row].as_matrix()
    for position in j:
        if position not in all_positions:
            all_positions.append(position)

/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
all_positions = all_positions.sort()

TypeError: '<' not supported between instances of 'NoneType' and 'str'

In [36]:
all_positions.remove(None)

In [39]:
all_positions.pop(-2)

nan

In [41]:
for i in all_positions:
    df_2019[i] = None

In [50]:
len(all_positions)

24

In [52]:
temp1.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Designer,"Developer, back-end","Developer, front-end","Developer, full-stack",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"Developer, full-stack",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Engineer, data",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Database administrator,"Developer, back-end","Developer, front-end","Developer, full-stack","Developer, QA or test",DevOps specialist,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Developer, back-end","Developer, desktop or enterprise applications","Developer, front-end","Developer, full-stack","Developer, game or graphics",Educator,None,...,None,None,None,None,None,None,None,None,None,None


In [53]:
temp1.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Designer,"Developer, back-end","Developer, front-end","Developer, full-stack",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"Developer, full-stack",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Engineer, data",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Database administrator,"Developer, back-end","Developer, front-end","Developer, full-stack","Developer, QA or test",DevOps specialist,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Developer, back-end","Developer, desktop or enterprise applications","Developer, front-end","Developer, full-stack","Developer, game or graphics",Educator,None,...,None,None,None,None,None,None,None,None,None,None


In [56]:
for i in range(temp1.shape[0]):
    j = temp1.iloc[i].as_matrix()
    for item in j:
        if item!=None:
            df_2019.iloc[i][item] = 1
        

/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [58]:
df_2019[all_positions].head()

,Designer,"Developer, back-end","Developer, front-end","Developer, full-stack",Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Engineer, data","Developer, QA or test",DevOps specialist,...,"Engineer, site reliability",Engineering manager,Senior executive/VP,System administrator,Student,"Developer, embedded applications or devices",Product manager,Academic researcher,Scientist,Marketing or sales professional
0,1,1,1,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,1,1,1,1,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,1,1,1,None,None,1,None,1,1,...,None,None,None,None,None,None,None,None,None,None
4,None,1,1,1,1,1,1,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [64]:
df_2019[all_positions] = df_2019[all_positions].fillna(value=0)

In [66]:
df_2019[all_positions].head()

,Designer,"Developer, back-end","Developer, front-end","Developer, full-stack",Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Engineer, data","Developer, QA or test",DevOps specialist,...,"Engineer, site reliability",Engineering manager,Senior executive/VP,System administrator,Student,"Developer, embedded applications or devices",Product manager,Academic researcher,Scientist,Marketing or sales professional
0,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
